# Init Data Warehouse for NYC Trip Data

### Year Start: 2014

In [1]:
# Import
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [2]:
# Start Spark Session
spark = SparkSession \
    .builder \
    .appName("Warehouse Init Job") \
    .getOrCreate()

24/12/28 05:14:25 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
# Path lists
zone_lookup = "hdfs://10.128.0.59:8020/raw_data/updated_zone_lookup.csv"
input_path = "hdfs://10.128.0.59:8020/raw_data/2014"

output_fact_trip = "hdfs://10.128.0.59:8020/data_warehouse/fact_trip"
output_dim_vendor = "hdfs://10.128.0.59:8020/data_warehouse/dim_vendor"
output_dim_datetime = "hdfs://10.128.0.59:8020/data_warehouse/dim_datetime"
output_dim_rate_code = "hdfs://10.128.0.59:8020/data_warehouse/dim_rate_code"
output_dim_pickup_location = "hdfs://10.128.0.59:8020/data_warehouse/dim_pickup_location"
output_dim_dropoff_location = "hdfs://10.128.0.59:8020/data_warehouse/dim_dropoff_location"
output_dim_payment = "hdfs://10.128.0.59:8020/data_warehouse/dim_payment"

### Input

In [4]:
# Schema
input_schema = StructType([
    StructField("VendorID", LongType(), True),
    StructField("tpep_pickup_datetime", TimestampNTZType(), True),
    StructField("tpep_dropoff_datetime", TimestampNTZType(), True),
    StructField("passenger_count", LongType(), True),
    StructField("trip_distance", DoubleType(), True),
    StructField("RatecodeID", LongType(), True),
    StructField("store_and_fwd_flag", StringType(), True),
    StructField("PULocationID", LongType(), True),
    StructField("DOLocationID", LongType(), True),
    StructField("payment_type", LongType(), True),
    StructField("fare_amount", DoubleType(), True),
    StructField("extra", DoubleType(), True),
    StructField("mta_tax", DoubleType(), True),
    StructField("tip_amount", DoubleType(), True),
    StructField("tolls_amount", DoubleType(), True),
    StructField("total_amount", DoubleType(), True)
])

lookup_schema = StructType([
    StructField("LocationID", LongType(), True),
    StructField("X", DoubleType(), True),
    StructField("Y", DoubleType(), True),
    StructField("zone", StringType(), True),
    StructField("borough", StringType(), True),
    StructField("service_zone", StringType(), True)
])

In [5]:
df_input = spark.read.format("parquet") \
    .schema(input_schema) \
    .load(input_path) \
    .dropna() \
    .filter((year(col("tpep_pickup_datetime")) == 2014) & \
            (col("trip_distance") > 0.0) & \
            (col("passenger_count") > 0)) \
    .withColumn("trip_id", monotonically_increasing_id())

df_lookup = spark.read.format("csv") \
    .schema(lookup_schema) \
    .option("header", True) \
    .load(zone_lookup) \
    .dropna()

df_input.printSchema()
df_lookup.printSchema()

# df_lookup.show(5)

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- trip_id: long (nullable = false)

root
 |-- LocationID: long (nullable = true)
 |-- X: double (nullable = true)
 |-- Y: double (nullable = true)
 |-- zone: string (nullable = true)
 |-- borough: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [6]:
# df_input.cache() - not enough RAM
# df_input.show(5, truncate = False)

### Output

In [7]:
# Datetime dimension
dim_datetime = df_input \
    .select("tpep_pickup_datetime", "tpep_dropoff_datetime") \
    .distinct() \
    .withColumn("datetime_id", monotonically_increasing_id()) \
    .withColumn("pick_hour", hour(col("tpep_pickup_datetime")) + minute(col("tpep_pickup_datetime")) / 60.0) \
    .withColumn("pick_day", dayofmonth(col("tpep_pickup_datetime"))) \
    .withColumn("pick_month", month(col("tpep_pickup_datetime"))) \
    .withColumn("pick_year", year(col("tpep_pickup_datetime"))) \
    .withColumn("pick_weekday", F.date_format(col("tpep_pickup_datetime"), "EEEE")) \
    .withColumn("pick_weekday_id", dayofweek(col("tpep_pickup_datetime"))) \
    .withColumn("drop_hour", hour(col("tpep_dropoff_datetime")) + minute(col("tpep_dropoff_datetime")) / 60.0) \
    .withColumn("drop_day", dayofmonth(col("tpep_dropoff_datetime"))) \
    .withColumn("drop_month", month(col("tpep_dropoff_datetime"))) \
    .withColumn("drop_year", year(col("tpep_dropoff_datetime"))) \
    .withColumn("drop_weekday", F.date_format(col("tpep_dropoff_datetime"), "EEEE")) \
    .withColumn("drop_weekday_id", dayofweek(col("tpep_dropoff_datetime")))

In [8]:
# Rate code dimension
rate_code_type = {
    1: "Standard rate",
    2: "JFK",
    3: "Newark",
    4: "Nassau or Westchester",
    5: "Negotiated fare",
    6: "Group ride",
}

# RatecodeID + rate_code_name
dim_rate_code = df_input \
    .select("RatecodeID") \
    .distinct() \
    .withColumn("rate_code_name",
                F.when(F.col("RatecodeID") == 1, rate_code_type[1])
                 .when(F.col("RatecodeID") == 2, rate_code_type[2])
                 .when(F.col("RatecodeID") == 3, rate_code_type[3])
                 .when(F.col("RatecodeID") == 4, rate_code_type[4])
                 .when(F.col("RatecodeID") == 5, rate_code_type[5])
                 .when(F.col("RatecodeID") == 6, rate_code_type[6])
                 .otherwise(None)
                ) \
    .dropna()

In [9]:
# Pickup location dimension
# PULocationID + Borough + Zone + service_zone
dim_pickup_location = df_input \
    .select("PULocationID") \
    .distinct() \
    .join(df_lookup, df_input.PULocationID == df_lookup.LocationID, "inner") \
    .select("PULocationID", "X", "Y", "zone", "borough", "service_zone")

# Dropoff location dimension
# DOLocationID + Borough + Zone + service_zone
dim_dropoff_location = df_input \
    .select("DOLocationID") \
    .distinct() \
    .join(df_lookup, df_input.DOLocationID == df_lookup.LocationID, "inner") \
    .select("DOLocationID", "X", "Y", "zone", "borough", "service_zone")

In [10]:
# Payment type dim
payment_type_name = {
    1: "Credit card",
    2: "Cash",
    3: "No charge",
    4: "Dispute",
    5: "Unknown",
    6: "Voided trip",
}
# payment_type + payment_type_name
dim_payment = df_input \
    .select("payment_type") \
    .distinct() \
    .withColumn("payment_type_name",
                F.when(F.col("payment_type") == 1, payment_type_name[1])
                 .when(F.col("payment_type") == 2, payment_type_name[2])
                 .when(F.col("payment_type") == 3, payment_type_name[3])
                 .when(F.col("payment_type") == 4, payment_type_name[4])
                 .when(F.col("payment_type") == 5, payment_type_name[5])
                 .when(F.col("payment_type") == 6, payment_type_name[6])
                 .otherwise(None)
                ) \
    .dropna()

# dim_payment.printSchema()
# dim_payment.distinct().show()

In [11]:
# Vendor dimension
vendor_name = {
    1: "Create Mobile Technologies, LLC",
    2: "VeriFone Inc",
}
# VendorID + vendor_name
dim_vendor = df_input \
    .select("VendorID") \
    .distinct() \
    .withColumn("vendor_name",
                F.when(F.col("VendorID") == 1, vendor_name[1])
                 .when(F.col("VendorID") == 2, vendor_name[2])
                 .otherwise(None)
                ) \
    .dropna()

In [12]:
# Fact table
fact_trip = df_input.alias("fact_data") \
    .join(dim_datetime.alias("dim_datetime"), (col("fact_data.tpep_pickup_datetime") == col("dim_datetime.tpep_pickup_datetime")) & (col("fact_data.tpep_dropoff_datetime") == col("dim_datetime.tpep_dropoff_datetime")), "inner") \
    .select(
        col("fact_data.trip_id"),
        col("fact_data.VendorID").alias("vendor_id"),
        col("dim_datetime.datetime_id").alias("datetimestamp_id"),
        col("fact_data.PULocationID").alias("pu_location_id"),
        col("fact_data.DOLocationID").alias("do_location_id"),
        col("fact_data.RatecodeID").alias("ratecode_id").cast("long"),
        col("fact_data.payment_type").alias("payment_id").cast("long"),
        col("fact_data.passenger_count"),
        col("fact_data.trip_distance"),
        col("fact_data.fare_amount"),
        col("fact_data.extra"),
        col("fact_data.mta_tax"),
        col("fact_data.tip_amount"),
        col("fact_data.tolls_amount"),
        col("fact_data.total_amount")
        # delete cause of version conflict
        # df.congestion_surcharge,
        # df.Airport_fee,
        # df.improvement_surcharge,
)

### Write to HDFS

In [13]:
# fact_trip_partitioned = fact_trip.coalesce(24)

fact_trip.write \
    .format("parquet") \
    .option("path", output_fact_trip) \
    .mode("overwrite") \
    .save()

In [14]:
# SCD Type II
# dim_datetime_partitioned = dim_datetime.coalesce(24)
dim_datetime.write \
    .partitionBy("pick_year", "pick_month") \
    .format("parquet") \
    .option("path", output_dim_datetime) \
    .mode("overwrite") \
    .save()

In [15]:
# SCD Type I
dim_vendor.write \
    .partitionBy("VendorID") \
    .format("parquet") \
    .option("path", output_dim_vendor) \
    .mode("overwrite") \
    .save()

In [16]:
# SCD Type I
dim_rate_code.write \
    .partitionBy("RatecodeID") \
    .format("parquet") \
    .option("path", output_dim_rate_code) \
    .mode("overwrite") \
    .save()

In [17]:
# SCD Type II
# pickup = dim_pickup_location.coalesce(12)

dim_pickup_location.write \
    .partitionBy("service_zone") \
    .format("parquet") \
    .option("path", output_dim_pickup_location) \
    .mode("overwrite") \
    .save()

In [18]:
# SCD Type II
# dropoff = dim_dropoff_location.coalesce(12)

dim_dropoff_location.write \
    .partitionBy("service_zone") \
    .format("parquet") \
    .option("path", output_dim_dropoff_location) \
    .mode("overwrite") \
    .save()

In [19]:
# SCD Type I
dim_payment.write \
    .partitionBy("payment_type") \
    .format("parquet") \
    .option("path", output_dim_payment) \
    .mode("overwrite") \
    .save()

### Stop Job (Must)

In [20]:
spark.stop()